# HASHCODE 2018 Qualification
## Self-driving rides
https://hashcode.withgoogle.com/past_editions.html <br>
Team: Cool Shenanigans <br>
21 milion points in the qualification round due to a minor bug... :( <br>
45 milion points in the extended qualification round ( where we fixed the bug, hoorraaay)

We worken in a Collaboratory notebook, also prvided by Google.

In our method we sorted the rides in order of their earliest start time. And then one by one we assigned the rides to the cars in the following way: <br>
-For each ride:
* If we can deliver in time
* * If we can take the bonus, then assign the ride to the car that is the closest to the ride start and can take the bonus and deliver in time
* * Else we cannot take the bonus, then assign ithe ride to the car that is the closest to the ride start and can deliver in time
* Else skip the ride

In [0]:
#importing data rom google sheets
!pip install --upgrade -q gspread
!pip install -U -q PyDrive

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters

#ListFile({'q': "title contains '.txt' and 'root' in parents"})
listed = drive.ListFile({'q': "title contains '.in' and 'root' in parents"}).GetList()

files_names = ['a_example.in', 'b_should_be_easy.in', 'c_no_hurry.in', 'd_metropolis.in', 'e_high_bonus.in']
files_ids = [0, 0, 0, 0, 0]
for file in listed:
    if files_names[0] in file['title']:
        files_ids[0] = file['id']
    elif files_names[1] in file['title']:
        files_ids[1] = file['id']
    elif files_names[2] in file['title']:
        files_ids[2] = file['id']
    elif files_names[3] in file['title']:
        files_ids[3] = file['id']
    elif files_names[4] in file['title']:
        files_ids[4] = file['id']

In [3]:
files_local_names = ['InputA.txt', 'InputB.txt', 'InputC.txt', 'InputD.txt', 'InputE.txt']
for idx,file_id in enumerate(files_ids):
  file3 = drive.CreateFile({'id': file_id})
  print('Downloading file %s from Google Drive' % file3['title']) # 'YandexRelPredChallenge.txt.png'
  file3.GetContentFile(files_local_names[idx])  # Save Drive file as a local file
  print(file3)

GoogleDriveFile({'id': '16IZrQKtk0ayTk5z6Pe2sGfIClNbQUk_O', 'kind': 'drive#file', 'etag': '"z_YVnINIY56ukYDqXpOL80gtv44/MTUxOTkyNjU2NTE0MA"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/16IZrQKtk0ayTk5z6Pe2sGfIClNbQUk_O', 'webContentLink': 'https://drive.google.com/uc?id=16IZrQKtk0ayTk5z6Pe2sGfIClNbQUk_O&export=download', 'alternateLink': 'https://drive.google.com/file/d/16IZrQKtk0ayTk5z6Pe2sGfIClNbQUk_O/view?usp=drivesdk', 'embedLink': 'https://drive.google.com/file/d/16IZrQKtk0ayTk5z6Pe2sGfIClNbQUk_O/preview?usp=drivesdk', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/application/octet-stream', 'title': 'a_example.in', 'mimeType': 'application/octet-stream', 'labels': {'starred': False, 'hidden': False, 'trashed': False, 'restricted': False, 'viewed': True}, 'createdDate': '2018-03-01T17:49:25.140Z', 'modifiedDate': '2018-03-01T17:49:25.140Z', 'modifiedByMeDate': '2018-03-01T17:49:25.140Z', 'lastViewedByMeDate': '2018-03-01T17:49:25.140Z', 'markedView

In [4]:
!ls

datalab  InputA.txt  InputB.txt  InputC.txt  InputD.txt  InputE.txt


In [0]:
class IOObject:
  
  @staticmethod
  def  get_input(file_name):
      inputs = get_file_data(file_name)
      rows, columns, cars, rides = process_input(inputs)
      return rows, columns, cars, rides
  
  @staticmethod
  def get_file_data(file_name):
      actions = []
      with open(file_name) as fd:
          for line in fd:
              actions.append(line.strip().split('\t'))
      return actions
  
  @staticmethod
  def process_input(input):
    fr = input[0][0].split()
    other = input[1:]

    R = fr[0] #rows
    C = fr[1] #columns
    F = fr[2] #number vehicles
    N = fr[3] #number rides
    B = fr[4] #ride bonuses
    T = fr[5] #total time

    rides = []
    cars = []
    counter = 0

    idx = 0
    for row in other:
      row = [int(x) for x in row[0].split()]
      start = Coordinate(row[0], row[1])
      end = Coordinate(row[2], row[3])
      ride = Ride(start, end, row[4], row[5], B, idx)
      idx += 1

      rides.append(ride)


    for i in range(int(F)):
      car = Car();
      cars.append(car)

    return R, C, cars, rides

In [0]:
class Ride:
    start = None #type: Coordinate
    end = None #type: Coordinate
    earliestStart = None # int
    latestStart = None # int 
    latestEnd = None # int
    bonusPoints = None
    duration = None
    ride_id = None
    
    def __init__(self, start, end, earliestStart, latestEnd, bonusPoints, ride_id):
        self.start = start
        self.end = end
        self.earliestStart = earliestStart
        self.latestEnd = latestEnd
        self.bonusPoints = int(bonusPoints)
        self.duration = self.start.calculateDistance(self.end)
        
        self.latestStart = self.latestEnd - self.duration
        
        self.ride_id = ride_id
   
    def __str__(self):
        #return 'ride id: '+str(self.ride_id) + ', start: ' + str(self.start) + ', end: ' + str(self.end) + ', earliest start: ' + str(self.earliestStart) + ', latest end: ' + str(self.latestEnd) + ', latest start: ' + str(self.latestStart)
        return 'ride id: '+str(self.ride_id) + ', duration: ' + str(self.duration) + ', earliest start: ' + str(self.earliestStart) + ', latest end: ' + str(self.latestEnd) + ', latest start: ' + str(self.latestStart) + ', start: ' + str(self.start) + ', end: ' + str(self.end)

In [0]:
class Car:
    currentPosition = None #type: Coordinate
    assignedRides = None #type: List[int] list of ride_id
    freeAtTime = None #type: int ;time when free
    
    def __init__(self):
        self.currentPosition = Coordinate(0, 0)
        self.assignedRides = []
        self.freeAtTime = 0
        
    def add_ride(self,ride : Ride):
        self.assignedRides.append(ride)
        
        distance_to = self.calculate_time_to_start(ride.start)
        time_of_start = self.freeAtTime
        if time_of_start + distance_to <= ride.earliestStart:
            start = ride.earliestStart
        else:
            start = time_of_start + distance_to
        self.freeAtTime = start + ride.duration
        
        self.currentPosition = ride.end
        
    def calculate_time_to_start(self, start):
        return self.currentPosition.calculateDistance(start)
      
    
    def __str__(self):
        return 'cur pos: ' + str(self.currentPosition) + ', free@: '+str(self.freeAtTime) + ', assignedRides: ' + \
          str([str(ride) for ride in self.assignedRides])

In [0]:
class Coordinate:
    x = None
    y = None


    def __init__(self, x, y):
        self.x = x
        self.y = y

    def calculateDistance(self, other):
        return abs(self.x - other.x) + abs(self.y - other.y)
      
    def __str__(self):
        return '('+str(self.x)+','+str(self.y)+')'

In [228]:
def binary_search_leftmost_index(array, value, start, stop):
    while start < stop:
        mid = int((start + stop) / 2)
        if value <= array[mid]:
            stop = mid
        else:
            start = mid + 1
    return start

a = [1,3,5]
ordered_result = []
for i in range(8):
  ordered_result.append(binary_search_leftmost_index(a, i, 0, len(a)))
print(ordered_result)

[0, 0, 1, 1, 2, 2, 3, 3]


In [0]:
class CarsRepo:
    def __init__(self, cars_list):
        #give them at moment 0 when they are all 0
        self.cars = cars_list
    
    def add_ride_to_car(self, ride):
        #type ride: Ride
        
        # if BONUS possible, car that arrives to start position the closest in time to the start time of the ride
        # else, choose car that arrives to start position as soon as possible
        # if this is still too l1ate and that wouldn't give any points return
        
        start_position = ride.start
        start_time = ride.earliestStart
        latest_start = ride.latestStart
        
        bonus_closest_car = None
        nonbonus_closest_car = None
        best_time_arrival = None
        #best_time_arrival: 0 if the car can arrive on time or before;
        # the time from start_time to car arrival time otherwise
        for car in self.cars:
#           #Crazy with closest to 0
#             arrival_time = car.freeAtTime + start_position.calculateDistance(car.currentPosition)
#             if arrival_time <= start_time:
#                 if bonus_closest_car != None:
#                     if best_time_arrival < arrival_time:
#                         best_time_arrival = arrival_time
#                         bonus_closest_car = car
#                 else:
#                     bonus_closest_car = car
#                     best_time_arrival = 0
#             else:
#                 candidate = arrival_time - start_time 
#                 if arrival_time <= latest_start and (best_time_arrival is None or candidate < best_time_arrival):
#                     best_time_arrival = candidate
#                     nonbonus_closest_car = car
            
            #assign to closest car
            arrival_time = car.freeAtTime + start_position.calculateDistance(car.currentPosition)
            candidate = start_position.calculateDistance(car.currentPosition)
    
            if arrival_time <= start_time:
                if bonus_closest_car != None:
                    if candidate < best_time_arrival:
                        best_time_arrival = arrival_time
                        bonus_closest_car = car
                else:
                    bonus_closest_car = car
                    best_time_arrival = candidate
            else:
                if arrival_time <=latest_start and (best_time_arrival is None or candidate < best_time_arrival):
                    best_time_arrival = candidate
                    nonbonus_closest_car = car
       
        if bonus_closest_car is not None:
            bonus_closest_car.add_ride(ride)
        elif nonbonus_closest_car is not None:
            nonbonus_closest_car.add_ride(ride)
   
    def __str__(self):
        r = ""
        
        for idx,car in enumerate(self.cars):
            #r += str(len(car.assignedRides)) + " " + " ".join([str(x) for x in car.assignedRides]) + "\n"
            r += str(car) + '\n'
        return r

In [0]:
class RidesRepo:
    def __init__(self):
        self.rides = []
    
    def bs_left_index_rides(self, time, start, stop):
        #gets position in array closest to time 
        while start < stop:
            mid = int((start + stop) / 2)
            if time <= self.rides[mid].earliestStart:
                stop = mid
            else:
                start = mid + 1
        return start
            
    def add_ride(self, ride):
        #Modify earliest_start to latest_start
        #insertion_idx = self.bs_left_index_rides(ride.latestStart, 0, len(self.rides))
        
        
        insertion_idx = self.bs_left_index_rides(ride.earliestStart, 0, len(self.rides))

#        aux_idx = insertion_idx    
#         while aux_idx < len(self.rides) - 1 and self.rides[aux_idx].earliestStart == self.rides[insertion_idx].earliestStart and self.rides[aux_idx].duration > self.rides[aux_idx + 1].duration:
#             aux_idx += 1
#         insertion_idx = aux_idx
        
        if insertion_idx == len(self.rides):
            self.rides.append(ride)
        else:
            self.rides.insert(insertion_idx, ride)
    
    def __str__(self):
        s = ""
        for ride in self.rides:
            s += str(ride) + '\n'
        return s

In [0]:
class Test:
    
    @staticmethod
    def test_cars_can_deliver_in_time(carsRepo):
        for car in carsRepo.cars:
            assert Test.check_assigned_list(car) == 1
        #print('Tests passed')
    
    @staticmethod
    def check_assigned_list(car):
        # do a thing like in the simulation
        last_pos = Coordinate(0, 0)
        time = 0
        success = 1
        for ride in car.assignedRides:
            
            distance_to = last_pos.calculateDistance(ride.start)
            if time + distance_to <= ride.earliestStart:
                start = ride.earliestStart
            else:
                start = time + distance_to
            
            distance_of = ride.duration

            if start + distance_of > ride.latestEnd:
                success = 0

            time = start + distance_of
            last_pos = ride.end
        return success

In [0]:
if __name__ == "__main__":
    #READ
    rows, columns, cars, rides = get_input(files_local_names[0])
    
    
    #INITIALIZE
    ridesRepo = RidesRepo()
    for ride in rides:
        ridesRepo.add_ride(ride)
    
    #print(ridesRepo)
    carsRepo = CarsRepo(cars)
    
    
    #SOLVE
    for ride in ridesRepo.rides:
        carsRepo.add_ride_to_car(ride)
        #print(carsRepo)
    
    #TEST
    Test.test_cars_can_deliver_in_time(carsRepo)
    
    #RESULT
    for car in carsRepo.cars:
        print(str(len(car.assignedRides)) + ' ' + ' '.join([str(ride.ride_id) for ride in car.assignedRides]))